<a href="https://colab.research.google.com/github/Prathambiradr12345/Machine-_learning/blob/main/optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 1.7 MB/s eta 0:00:00


In [5]:
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [6]:
import pandas as pd
import numpy as np

In [7]:
df=pd.read_csv('diabetes.csv')

In [8]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [20]:
# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [21]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [23]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2025-01-27 06:33:02,654] A new study created in memory with name: no-name-ad5c51ae-d710-430f-afd3-ed16f2670a93
[I 2025-01-27 06:33:05,680] Trial 0 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 167, 'max_depth': 14}. Best is trial 0 with value: 0.7728119180633147.
[I 2025-01-27 06:33:07,052] Trial 1 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 144, 'max_depth': 15}. Best is trial 0 with value: 0.7728119180633147.
[I 2025-01-27 06:33:08,299] Trial 2 finished with value: 0.7616387337057727 and parameters: {'n_estimators': 127, 'max_depth': 4}. Best is trial 0 with value: 0.7728119180633147.
[I 2025-01-27 06:33:08,847] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 83, 'max_depth': 14}. Best is trial 0 with value: 0.7728119180633147.
[I 2025-01-27 06:33:09,292] Trial 4 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 79, 'max_depth': 7}. Best is trial 0 with value: 0.77281191

In [24]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 197, 'max_depth': 16}


In [25]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


In [26]:
#Samplers in Optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize



In [27]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-01-27 06:40:07,851] A new study created in memory with name: no-name-983ae18c-0095-478d-a722-26d0838aa4dd
[I 2025-01-27 06:40:08,579] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 85, 'max_depth': 20}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-01-27 06:40:09,099] Trial 1 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 107, 'max_depth': 4}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-01-27 06:40:09,701] Trial 2 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 107, 'max_depth': 19}. Best is trial 2 with value: 0.7746741154562384.
[I 2025-01-27 06:40:10,440] Trial 3 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 131, 'max_depth': 10}. Best is trial 2 with value: 0.7746741154562384.
[I 2025-01-27 06:40:11,004] Trial 4 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 99, 'max_depth': 13}. Best is trial 2 with value: 0.7746741

In [28]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7821229050279329
Best hyperparameters: {'n_estimators': 124, 'max_depth': 18}


In [29]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


In [39]:
#gradeSearch

search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [31]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-01-27 06:41:32,787] A new study created in memory with name: no-name-4a1aa571-0f01-4977-9b4e-13ca5de2f5a3
[I 2025-01-27 06:41:33,550] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-01-27 06:41:34,373] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-01-27 06:41:34,661] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-01-27 06:41:35,227] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-01-27 06:41:35,782] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [32]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [33]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')

Test Accuracy with best hyperparameters: 0.74


In [35]:
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances
# 1. Optimization History
plot_optimization_history(study).show()

In [36]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [37]:
# 3. Slice Plot
plot_slice(study).show()

In [38]:
# 4. Contour Plot
plot_contour(study).show()

In [40]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [41]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [42]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [43]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-01-27 07:55:55,406] A new study created in memory with name: no-name-74d15970-5d22-4d08-9a6d-2017991e590a
[I 2025-01-27 07:55:59,253] Trial 0 finished with value: 0.7392923649906891 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 207, 'learning_rate': 0.20143555400208107, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.7392923649906891.
[I 2025-01-27 07:56:01,454] Trial 1 finished with value: 0.750465549348231 and parameters: {'classifier': 'RandomForest', 'n_estimators': 291, 'max_depth': 20, 'min_samples_split': 2, 'min_samples_leaf': 6, 'bootstrap': True}. Best is trial 1 with value: 0.750465549348231.
[I 2025-01-27 07:56:02,461] Trial 2 finished with value: 0.7709497206703911 and parameters: {'classifier': 'RandomForest', 'n_estimators': 240, 'max_depth': 8, 'min_samples_split': 4, 'min_samples_leaf': 10, 'bootstrap': False}. Best is trial 2 with value: 0.7709497206703911.
[I 2025-01-27 07:56:04,757] Trial 3 

In [44]:
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.12339944670959323, 'kernel': 'linear', 'gamma': 'auto'}
Best trial accuracy: 0.7895716945996275


In [45]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.739292,2025-01-27 07:55:55.412668,2025-01-27 07:55:59.252827,0 days 00:00:03.840159,NaN,NaN,GradientBoosting,NaN,NaN,0.201436,15.0,9.0,2.0,207.0,COMPLETE
1,1,0.750466,2025-01-27 07:55:59.261611,2025-01-27 07:56:01.454513,0 days 00:00:02.192902,NaN,True,RandomForest,NaN,NaN,NaN,20.0,6.0,2.0,291.0,COMPLETE
2,2,0.770950,2025-01-27 07:56:01.456343,2025-01-27 07:56:02.460883,0 days 00:00:01.004540,NaN,False,RandomForest,NaN,NaN,NaN,8.0,10.0,4.0,240.0,COMPLETE
3,3,0.746741,2025-01-27 07:56:02.462642,2025-01-27 07:56:04.756835,0 days 00:00:02.294193,NaN,NaN,GradientBoosting,NaN,NaN,0.020759,14.0,4.0,6.0,136.0,COMPLETE
4,4,0.782123,2025-01-27 07:56:04.758558,2025-01-27 07:56:04.802390,0 days 00:00:00.043832,0.368895,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.785847,2025-01-27 07:56:32.510766,2025-01-27 07:56:32.545837,0 days 00:00:00.035071,0.212400,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.703911,2025-01-27 07:56:32.547489,2025-01-27 07:56:32.589613,0 days 00:00:00.042124,0.100594,NaN,SVM,auto,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.789572,2025-01-27 07:56:32.592309,2025-01-27 07:56:32.632189,0 days 00:00:00.039880,0.153249,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.748603,2025-01-27 07:56:32.633779,2025-01-27 07:56:34.142718,0 days 00:00:01.508939,NaN,NaN,GradientBoosting,NaN,NaN,0.023637,16.0,5.0,3.0,74.0,COMPLETE


In [46]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,78
GradientBoosting,11
RandomForest,11


In [47]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.743525
RandomForest,0.764686
SVM,0.774674


In [48]:
# 1. Optimization History
plot_optimization_history(study).show()

In [49]:
# 3. Slice Plot
plot_slice(study).show()

In [50]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [53]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


[I 2025-01-27 08:01:30,603] A new study created in memory with name: no-name-3f20c0e7-8413-4bc2-a783-ca4a499923dc


[0]	train-mlogloss:0.97367	eval-mlogloss:0.96798
[1]	train-mlogloss:0.88548	eval-mlogloss:0.85904
[2]	train-mlogloss:0.80290	eval-mlogloss:0.78052
[3]	train-mlogloss:0.69296	eval-mlogloss:0.66828
[4]	train-mlogloss:0.62990	eval-mlogloss:0.60513
[5]	train-mlogloss:0.56652	eval-mlogloss:0.54009
[6]	train-mlogloss:0.51730	eval-mlogloss:0.48981
[7]	train-mlogloss:0.51069	eval-mlogloss:0.48295
[8]	train-mlogloss:0.51008	eval-mlogloss:0.48346
[9]	train-mlogloss:0.50568	eval-mlogloss:0.47496
[10]	train-mlogloss:0.48817	eval-mlogloss:0.45135
[11]	train-mlogloss:0.48750	eval-mlogloss:0.45171
[12]	train-mlogloss:0.46768	eval-mlogloss:0.43199
[13]	train-mlogloss:0.46685	eval-mlogloss:0.43098
[14]	train-mlogloss:0.46470	eval-mlogloss:0.42686
[15]	train-mlogloss:0.46329	eval-mlogloss:0.42422
[16]	train-mlogloss:0.46220	eval-mlogloss:0.42245
[17]	train-mlogloss:0.45568	eval-mlogloss:0.41504
[18]	train-mlogloss:0.43722	eval-mlogloss:0.39355
[19]	train-mlogloss:0.43565	eval-mlogloss:0.39184
[20]	train

[I 2025-01-27 08:01:41,227] Trial 0 finished with value: 1.0 and parameters: {'lambda': 1.5967143623093272e-06, 'alpha': 1.906697964462894e-05, 'eta': 0.16422713836347963, 'gamma': 0.0027833797644522614, 'max_depth': 8, 'min_child_weight': 9, 'subsample': 0.4479939758765569, 'colsample_bytree': 0.816949409704016}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:0.81664	eval-mlogloss:0.79412
[1]	train-mlogloss:0.66626	eval-mlogloss:0.62522
[2]	train-mlogloss:0.52660	eval-mlogloss:0.47500
[3]	train-mlogloss:0.41831	eval-mlogloss:0.35996
[4]	train-mlogloss:0.35722	eval-mlogloss:0.30351
[5]	train-mlogloss:0.30317	eval-mlogloss:0.24367
[6]	train-mlogloss:0.26561	eval-mlogloss:0.20424
[7]	train-mlogloss:0.25447	eval-mlogloss:0.19288
[8]	train-mlogloss:0.25536	eval-mlogloss:0.19618
[9]	train-mlogloss:0.24606	eval-mlogloss:0.18109
[10]	train-mlogloss:0.24227	eval-mlogloss:0.17878
[11]	train-mlogloss:0.23455	eval-mlogloss:0.16854
[12]	train-mlogloss:0.23341	eval-mlogloss:0.16759
[13]	train-mlogloss:0.23422	eval-mlogloss:0.16972
[14]	train-mlogloss:0.23349	eval-mlogloss:0.16803
[15]	train-mlogloss:0.23195	eval-mlogloss:0.16643
[16]	train-mlogloss:0.23066	eval-mlogloss:0.16487
[17]	train-mlogloss:0.23052	eval-mlogloss:0.16599
[18]	train-mlogloss:0.22286	eval-mlogloss:0.15693
[19]	train-mlogloss:0.22138	eval-mlogloss:0.15587
[20]	train

[I 2025-01-27 08:01:45,639] Trial 1 finished with value: 1.0 and parameters: {'lambda': 1.305978477815269e-07, 'alpha': 1.1057829056007313e-05, 'eta': 0.24631131187268465, 'gamma': 7.386616265342189e-07, 'max_depth': 5, 'min_child_weight': 5, 'subsample': 0.43343074913958024, 'colsample_bytree': 0.8464354286245666}. Best is trial 0 with value: 1.0.


...

[0]	train-mlogloss:0.94158	eval-mlogloss:0.92731
[1]	train-mlogloss:0.81506	eval-mlogloss:0.79824


[I 2025-01-27 08:02:05,699] Trial 48 pruned. Trial was pruned at iteration 2.


[0]	train-mlogloss:1.05601	eval-mlogloss:1.05354
[1]	train-mlogloss:1.02427	eval-mlogloss:1.01788
[2]	train-mlogloss:0.98664	eval-mlogloss:0.97973
[3]	train-mlogloss:0.94701	eval-mlogloss:0.93842
[4]	train-mlogloss:0.91309	eval-mlogloss:0.90313
[5]	train-mlogloss:0.87858	eval-mlogloss:0.86698
[6]	train-mlogloss:0.85279	eval-mlogloss:0.83942
[7]	train-mlogloss:0.82551	eval-mlogloss:0.81366
[8]	train-mlogloss:0.80244	eval-mlogloss:0.79089


[I 2025-01-27 08:02:05,824] Trial 49 pruned. Trial was pruned at iteration 8.


Best trial: {'lambda': 1.5967143623093272e-06, 'alpha': 1.906697964462894e-05, 'eta': 0.16422713836347963, 'gamma': 0.0027833797644522614, 'max_depth': 8, 'min_child_weight': 9, 'subsample': 0.4479939758765569, 'colsample_bytree': 0.816949409704016}
Best accuracy: 1.0


In [52]:
!pip install optuna-integration[xgboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 2.8 MB/s eta 0:00:00


In [54]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()